In [1]:
#Calculating RFM

In [2]:
import pandas as pd
import datetime as dt

In [3]:
df = pd.read_csv("b2c.csv", parse_dates = ["InvoiceDate"])

In [4]:
snapshot_date = df["InvoiceDate"].max()+dt.timedelta(days=1)

In [5]:
snapshot_date

Timestamp('2011-12-10 12:50:00')

In [6]:
df = df.groupby("Customer ID").agg({
            "InvoiceDate": lambda date : (snapshot_date - date.max()).days,
            "Invoice" : "nunique",
            "Price" : "sum"})

In [7]:
df

,InvoiceDate,Invoice,Price
Customer ID,,,
12346,529,11,206.36
12347,2,8,554.57
12348,75,5,193.10
12349,19,4,1480.44
12350,310,1,65.30
...,...,...,...
18283,4,22,1651.60
18284,432,1,91.09
18285,661,1,100.20


In [8]:
df.rename (columns = { "InvoiceDate" : "Recency",
                       "Invoice"     : "Frequency",
                        "Price"      : "Monetary"}, inplace=True )

In [9]:
df

,Recency,Frequency,Monetary
Customer ID,,,
12346,529,11,206.36
12347,2,8,554.57
12348,75,5,193.10
12349,19,4,1480.44
12350,310,1,65.30
...,...,...,...
18283,4,22,1651.60
18284,432,1,91.09
18285,661,1,100.20


In [10]:
# RFM Segmentation

In [11]:
df["R_Score"] = pd.qcut( df["Recency"], 4, labels=[4,3,2,1])
df["F_Score"] = pd.qcut( df["Frequency"].rank(method ="first"), 4,labels=[1,2,3,4])
df["M_Score"] = pd.qcut( df["Monetary"], 4, labels=[1,2,3,4])

In [12]:
df

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score
Customer ID,,,,,,
12346,529,11,206.36,1,4,3
12347,2,8,554.57,4,4,4
12348,75,5,193.10,3,3,3
12349,19,4,1480.44,4,3,4
12350,310,1,65.30,2,1,2
...,...,...,...,...,...,...
18283,4,22,1651.60,4,4,4
18284,432,1,91.09,1,2,2
18285,661,1,100.20,1,2,2


In [13]:
df["RFM Score"] = df["R_Score"].astype(str) + df["F_Score"].astype(str) + df["M_Score"].astype(str)

In [14]:
df

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM Score
Customer ID,,,,,,,
12346,529,11,206.36,1,4,3,143
12347,2,8,554.57,4,4,4,444
12348,75,5,193.10,3,3,3,333
12349,19,4,1480.44,4,3,4,434
12350,310,1,65.30,2,1,2,212
...,...,...,...,...,...,...,...
18283,4,22,1651.60,4,4,4,444
18284,432,1,91.09,1,2,2,122
18285,661,1,100.20,1,2,2,122


In [15]:
seg_map= {

r'444' :"Champions" ,
r'34[1-4]' : "Loyal Customers",
r'4[2-4][1-4]' : "Potential Loyalists",
r'41[1-2]' : "New Customers",
r'3[1-3][1-4]' : "Needs Attention",
r'[1-2]4[1-4]' : "At Risk",
r'[1-2][1-2][1-4]' : "Hibernating"}


In [16]:
df["Segment"] = df["RFM Score"].replace (seg_map,regex= True)

In [22]:
df[["R_Score", "F_Score", "M_Score", "RFM Score", "Segment"]]